In [1]:
import torch
import logging
from typing import *

ModuleNotFoundError: No module named 'torch'

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
base_val = torch.as_tensor([
    [ #false
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
    ], [ #succ
        [0,1,0,0,0],
        [0,0,1,0,0],
        [0,0,0,1,0],
        [0,0,0,0,1],
        [0,0,0,0,0]
    ], [ #p
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
    ], [ #q
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
    ]
], dtype=torch.float)

In [ ]:

class Rulebook(NamedTuple):
    body_predicates : torch.Tensor
    variable_choices : torch.Tensor

    def to(self, device : torch.device):
        self.body_predicates = self.body_predicates.to(device)
        self.variable_choices = self.variable_choices.to(device)


In [ ]:
rulebook = Rulebook(
        body_predicates=torch.as_tensor([
            [[[0,0] for _ in range(0,11)] for _ in range(0,2)],
            [[[0,0] for _ in range(0,11)] for _ in range(0,2)],
            [[[1,1],[1,1],[1,1],[1,1],[3,3],[2,0],[1,1],[0,0],[1,2],[2,1],[2,2]] for _ in range(0,2)],
            [[[1,1],[1,1],[1,1],[1,1],[3,3],[2,0],[1,1],[0,0],[1,2],[2,1],[2,2]] for _ in range(0,2)],
        ]),
        variable_choices=torch.as_tensor([
            [[[0,0] for _ in range(0,11)] for _ in range(0,2)],
            [[[0,0] for _ in range(0,11)] for _ in range(0,2)],
            [[[0,0],[1,0],[2,7],[5,4],[3,3],[2,0],[1,1],[0,0],[1,2],[2,1],[2,2]] for _ in range(0,2)],
            [[[0,0],[1,0],[2,7],[5,4],[3,3],[2,0],[1,1],[0,0],[1,2],[2,1],[2,2]] for _ in range(0,2)],
        ])
    )

In [ ]:
pred_names = ['false', 'succ', 'p', 'q']
def var_choices(n : int, vars : int = 3) -> List[int]:
    return [int(n) // vars, n % vars]
def rule_str(rs : List[int], predicate : int, rulebook : Rulebook) -> str:
    lines = []
    for clause in range(0, rulebook.body_predicates.shape[1]):
        ret = []
        for i in range(0, rulebook.body_predicates.shape[3]):
            vs = ','.join(map(lambda v: chr(ord('A')+v),  var_choices(rulebook.variable_choices[predicate,clause,rs[clause],i])))
            ret.append(f'{pred_names[rulebook.body_predicates[predicate,clause,rs[clause],i]]}({vs})')
        lines.append(f"{pred_names[predicate]}(A, B) :- {','.join(ret)}")
    return '\n'.join(lines)

In [ ]:
print(rule_str([2,3], predicate=2, rulebook = rulebook))

p(A, B) :- succ(A,C),succ(C,B)
p(A, B) :- succ(B,C),succ(B,B)


In [ ]:
def extend_val(val : torch.Tensor, vars : int = 3) -> torch.Tensor:
    i = 0
    ret = []
    shape = list(val.shape) + [val.shape[-1] for _ in range(0, vars - 2)]
    valt = val.transpose(1, 2)
    for arg1 in range(0, vars):
        for arg2 in range(0, vars):
            v = val
            if arg1 == arg2:
                v = v.diagonal(dim1=1,dim2=2)
                for _ in range(0, arg1):
                    v = v.unsqueeze(1)
                while len(v.shape) < vars+1:
                    v = v.unsqueeze(-1)
            else:
                if arg2 < arg1:
                    v = valt
                unused = (x for x in range(0, vars) if x not in {arg1, arg2})
                for u in unused:
                    v = v.unsqueeze(u + 1)
            logging.debug(f"{i=} {arg1=} {arg2=} {v.shape=}")
            v = torch.broadcast_to(v, shape) #type: ignore
            v = v.unsqueeze(1)
            ret.append(v)
            i += 1
    return torch.cat(ret, dim=1)

In [ ]:
val2 = extend_val(base_val)
val2.shape

DEBUG:root:i=0 arg1=0 arg2=0 v.shape=torch.Size([4, 5, 1, 1])
DEBUG:root:i=1 arg1=0 arg2=1 v.shape=torch.Size([4, 5, 5, 1])
DEBUG:root:i=2 arg1=0 arg2=2 v.shape=torch.Size([4, 5, 1, 5])
DEBUG:root:i=3 arg1=1 arg2=0 v.shape=torch.Size([4, 5, 5, 1])
DEBUG:root:i=4 arg1=1 arg2=1 v.shape=torch.Size([4, 1, 5, 1])
DEBUG:root:i=5 arg1=1 arg2=2 v.shape=torch.Size([4, 1, 5, 5])
DEBUG:root:i=6 arg1=2 arg2=0 v.shape=torch.Size([4, 5, 1, 5])
DEBUG:root:i=7 arg1=2 arg2=1 v.shape=torch.Size([4, 1, 5, 5])
DEBUG:root:i=8 arg1=2 arg2=2 v.shape=torch.Size([4, 1, 1, 5])


torch.Size([4, 9, 5, 5, 5])

In [ ]:
bp = torch.arange(0,3, dtype=torch.long).repeat(30)
vc = torch.arange(0,9,dtype=torch.long).repeat(10)
bp.shape, vc.shape

(torch.Size([90]), torch.Size([90]))

In [ ]:
val2.shape, val2[bp,vc].shape
weights = torch.rand(2,2,90).softmax(-1)
weights = weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
val3 = val2[bp,vc].unsqueeze(0).unsqueeze(0)
print(val3.shape, weights.shape)
val4 = val3 * weights

torch.Size([1, 1, 90, 5, 5, 5]) torch.Size([2, 2, 90, 1, 1, 1])


In [ ]:
def disjuction2(a : torch.Tensor, b : torch.Tensor) -> torch.Tensor:
    return 1 - (1 - a) * (1 - b)

def disjunction_dim(a : torch.Tensor, dim : int = -1) -> torch.Tensor:
    return 1 - ((1 - a).prod(dim = dim))

In [ ]:

def conjuction2(a : torch.Tensor, b : torch.Tensor) -> torch.Tensor:
    return a * b
def conjunction_dim(a : torch.Tensor, dim : int = -1) -> torch.Tensor:
    return a.prod(dim=dim)

In [ ]:
disjuction2(0, 1)

1

In [ ]:
def infer_single_step(ex_val : torch.Tensor, rules : Rulebook, rule_weights : torch.Tensor) -> torch.Tensor:
    logging.debug(f"{ex_val.shape=} {rules.body_predicates.shape=} {rules.variable_choices.shape=}")
    ex_val = ex_val[rules.body_predicates, rules.variable_choices]
    logging.debug(f"{ex_val.shape=}")
    #conjuction of body predictes
    ex_val = conjunction_dim(ex_val, dim = 3)
    #existential quantification
    ex_val = disjunction_dim(ex_val, dim = -1)
    #rule weighing
    rule_weights = rule_weights.softmax(-1).unsqueeze(-1).unsqueeze(-1)
    ex_val = ex_val * rule_weights
    ex_val = ex_val.sum(dim = 2)
    #disjunction on clauses
    ex_val = disjunction_dim(ex_val, dim = 1)
    return ex_val

In [ ]:
infer_single_step(val2, rulebook2, torch.rand(4,2,11)).shape

DEBUG:root:ex_val.shape=torch.Size([4, 9, 5, 5, 5]) rules.body_predicates.shape=torch.Size([1, 2, 11, 2]) rules.variable_choices.shape=torch.Size([1, 2, 11, 2])
DEBUG:root:ex_val.shape=torch.Size([1, 2, 11, 2, 5, 5, 5])


torch.Size([4, 5, 5])

In [ ]:
val = base_val
weights = torch.rand(4, 2, 11)
        #predicate, clause, rule
for _ in range(0,5):
    val2 = extend_val(val)
    val2 = infer_single_step(val2, rulebook, weights)
    val = disjuction2(val, val2)
    del val2

DEBUG:root:i=0 arg1=0 arg2=0 v.shape=torch.Size([4, 5, 1, 1])
DEBUG:root:i=1 arg1=0 arg2=1 v.shape=torch.Size([4, 5, 5, 1])
DEBUG:root:i=2 arg1=0 arg2=2 v.shape=torch.Size([4, 5, 1, 5])
DEBUG:root:i=3 arg1=1 arg2=0 v.shape=torch.Size([4, 5, 5, 1])
DEBUG:root:i=4 arg1=1 arg2=1 v.shape=torch.Size([4, 1, 5, 1])
DEBUG:root:i=5 arg1=1 arg2=2 v.shape=torch.Size([4, 1, 5, 5])
DEBUG:root:i=6 arg1=2 arg2=0 v.shape=torch.Size([4, 5, 1, 5])
DEBUG:root:i=7 arg1=2 arg2=1 v.shape=torch.Size([4, 1, 5, 5])
DEBUG:root:i=8 arg1=2 arg2=2 v.shape=torch.Size([4, 1, 1, 5])
DEBUG:root:ex_val.shape=torch.Size([4, 9, 5, 5, 5]) rules.body_predicates.shape=torch.Size([4, 2, 11, 2]) rules.variable_choices.shape=torch.Size([4, 2, 11, 2])
DEBUG:root:ex_val.shape=torch.Size([4, 2, 11, 2, 5, 5, 5])
DEBUG:root:i=0 arg1=0 arg2=0 v.shape=torch.Size([4, 5, 1, 1])
DEBUG:root:i=1 arg1=0 arg2=1 v.shape=torch.Size([4, 5, 5, 1])
DEBUG:root:i=2 arg1=0 arg2=2 v.shape=torch.Size([4, 5, 1, 5])
DEBUG:root:i=3 arg1=1 arg2=0 v.shape

In [ ]:
val.shape

torch.Size([4, 5, 5])

In [ ]:
targets = torch.as_tensor([
        [2,0,2],
        [2,1,3],
        [2,2,4],
        [2,1,1],
        [2,3,2],
        [2,0,0],
        [2,0,1]
    ])
target_values = torch.as_tensor([
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,])


In [ ]:
def merge_pad(ts : List[torch.Tensor], dim : int = 0, newdim : int = 0) -> torch.Tensor:
    max_len = max(len(t) for t in ts)
    def required_shape(t : torch.Tensor) -> List[int]:
        ret = list(t.shape)
        ret[dim] = max_len - ret[dim]
        return ret
    ts = (torch.cat((t, torch.zeros(size=required_shape(t), device=t.device)), dim=dim).unsqueeze(newdim) for t in ts)
    return torch.cat(list(ts), dim=newdim)

In [ ]:
def merge_mask(ts : List[torch.Tensor], dim : int = 0, newdim : int = 0) -> torch.Tensor:
    max_len = max(len(t) for t in ts)
    def required_shape(t : torch.Tensor) -> List[int]:
        ret = list(t.shape)
        ret[dim] = max_len - ret[dim]
        return ret
    ts = (torch.cat((torch.zeros_like(t), 
        torch.as_tensor(-float('inf'), device=t.device).repeat(required_shape(t))), dim=dim).unsqueeze(newdim) for t in ts)
    return torch.cat(list(ts), dim=newdim)

In [ ]:
l = [[1,2],[3],[4,5,6]]
l = list(torch.as_tensor(x) for x in l)
merge_mask(l)

tensor([[0., 0., -inf],
        [0., -inf, -inf],
        [0., 0., 0.]])

In [ ]:
l